## Training data Generation

10 params to run the ROI generation script:
- h5path
- hiclip_val : Target channel, higher limit for clipping
- lowclip_val : Target channel, lower limit for clipping
- nuc_clip_low   : Nuc channel, lower limit for clipping
- nuc_clip_high  : Nuc channel, higher limit for clipping
- cyto_clip_low  : Cyto channel, lower limit for clipping
- cyto_clip_high : Cyto channel, higher limit for clipping
- xstart : x-coordinate for selected ROI
- ystart : y-coordinate for selected ROI
- zstart : z-coordinate for selected ROI

Return: 
* "Train" folder in current directory, with "Ch0", "Ch1", "Ch2" as subfolder, 
* Cropped ROIs blocks are saved in this folder w/ this format: *OTLS4_NODO_mmddyy_xxxxx_Xpos_xxxxx_xxxxx_Ypos_xxxxx_xxxxx_stack_xxxxx_xxxxx_mirror* 

In [1]:
import os
import h5py as h5
import glob
import matplotlib.pyplot as plt
from CollectImgStackFused import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
def main(h5path
        ,hiclip_val
        ,lowclip_val
        ,nuc_clip_low
        ,nuc_clip_high
        ,cyto_clip_low
        ,cyto_clip_high
        ,clahe
        ,xstart
        ,ystart
        ,zstart
        ):

    filename = h5path
    savedir = os.getcwd() + os.sep + 'train'
    blockname = filename.split("/")[3] + '_test'
    print(blockname)
    print(savedir)

    xstop = xstart + 640
    xcoords = (xstart, xstop)

    ystop = ystart + 640
    ycoords = (ystart, ystop)

    zstop = zstart + 50
    zcoords = (zstart, zstop)

    with h5.File(filename, 'r') as f:
        collectImgStackFused(f,
                        savedir,
                        blockname,
                        zcoords,
                        xcoords,
                        ycoords,
                        CLAHE=clahe,
                        hiclip_val  = hiclip_val,
                        lowclip_val = lowclip_val,
                        nuc_clip_low   = nuc_clip_low,
                        nuc_clip_high  = nuc_clip_high,
                        cyto_clip_low  = cyto_clip_low,
                        cyto_clip_high = cyto_clip_high)
    f.close()

**Manual Setup**

you could preview to find the coords with bigstitcher or GUI

In [ ]:
# change this
h5path = "W:/Trilabel_Data/PGP9.5/OTLS4_NODO_6-7-23_16-043J_PGP9.5/data-f0.h5"

hiclip_val = 2200
lowclip_val = 120
nuc_clip_low = 10
nuc_clip_high = 10000
cyto_clip_low = 10
cyto_clip_high = 5000
clahe = False

xstart = 200
ystart = 2500
zstart = 30

main(h5path
    ,hiclip_val,lowclip_val
    ,nuc_clip_low,nuc_clip_high
    ,cyto_clip_low,cyto_clip_high
    ,clahe
    ,xstart,ystart,zstart)

**Auto Set up with GUI** 

Using *ROI_v2.py*, export all the params into *ROI_coords_yyyy-mm-dd.csv* file

In [ ]:
import pandas as pd
fname = "Y:/Sarah/ROI_PreviewUI/ROI_coords_2023-07-14.csv" #change this
df = pd.read_csv(fname)
df = df.dropna(how="all")

for i in range(len(df)):
    h5path = df["h5path"][i]
    hiclip_val = df["pgp_clipHigh"][i]
    lowclip_val = df["pgp_clipLow"][i]
    nuc_clip_low = df["nuc_clipLow"][i]
    nuc_clip_high = df["nuc_clipHigh"][i]
    cyto_clip_low = df["cyto_clipLow"][i]
    cyto_clip_high = df["cyto_clipHigh"][i]
    if df["pgp_ctehmt_method"][i] == "Rescale":
        clahe = False
    else:
        clahe = True    
    xstart = df["xcoord"][i]
    ystart = df["ycoord"][i]
    zstart = df["CurrentZlevel"][i]

    main(h5path
        ,hiclip_val,lowclip_val
        ,nuc_clip_low,nuc_clip_high
        ,cyto_clip_low,cyto_clip_high
        ,clahe
        ,xstart,ystart,zstart)